In [24]:
import pandas as pd
import pickle

In [25]:
file_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codeT5_embeddings/df_dc_embeded_by_line_small.pkl'
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

/var/folders/d8/cr798cb57kzd5kz_r8g8jm380000gn/T/ipykernel_29567/1855975146.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(fh)


In [26]:
df.head()

,sample_id,severity,method,label,embeded_sequence_sum,embeded_sequence_avg
0,8077,critical,public class DruidPooledCallableStatement exte...,1,0 189.393524 1 -147.649734 2 -362...,0 0.203212 1 -0.158422 2 -0.38943...
1,5553,critical,public interface IAopReferenceModel {\n\t\n\tv...,1,0 3.520596 1 0.237391 2 -6.19085...,0 0.153069 1 0.010321 2 -0.26916...
2,9341,critical,@XmlAccessorType(XmlAccessType.FIELD)\n@XmlTyp...,1,0 3.876191 1 4.709641 2 -14.20...,0 0.104762 1 0.127288 2 -0.38399...
3,10419,critical,public class Ipam {\n private String driver;\...,1,0 20.053326 1 -9.636232 2 -25.69...,0 0.294902 1 -0.141709 2 -0.37787...
4,12232,critical,public class DruidPooledCallableStatement exte...,1,0 188.645142 1 -144.839706 2 -355...,0 0.202409 1 -0.155407 2 -0.38151...


In [27]:
embeded_sum = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_sum']]
embeded_avg = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_avg']]
embeded_sum.head()

,sample_id,severity,label,embeded_sequence_sum
0,8077,critical,1,0 189.393524 1 -147.649734 2 -362...
1,5553,critical,1,0 3.520596 1 0.237391 2 -6.19085...
2,9341,critical,1,0 3.876191 1 4.709641 2 -14.20...
3,10419,critical,1,0 20.053326 1 -9.636232 2 -25.69...
4,12232,critical,1,0 188.645142 1 -144.839706 2 -355...


In [28]:
print(embeded_sum["embeded_sequence_sum"].head())
print(embeded_sum["embeded_sequence_sum"].apply(type).value_counts())

0    0      189.393524
1     -147.649734
2     -362...
1    0      3.520596
1      0.237391
2     -6.19085...
2    0       3.876191
1       4.709641
2     -14.20...
3    0      20.053326
1      -9.636232
2     -25.69...
4    0      188.645142
1     -144.839706
2     -355...
Name: embeded_sequence_sum, dtype: object
embeded_sequence_sum
<class 'pandas.core.series.Series'>    3694
Name: count, dtype: int64


In [29]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [30]:
import re

def parse_flat_embedding(embedding_str):
    # Ensure input is a string
    if not isinstance(embedding_str, str):
        return []
    try:
        # Extract numerical values from the string
        values = re.findall(r"[-+]?\d*\.\d+|\d+", embedding_str)
        return [float(v) for v in values]
    except Exception as e:
        print(f"Error parsing: {embedding_str} -> {e}")
        return []


In [31]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(parse_flat_embedding)

In [32]:
n = len(embeded_sum["embeded_sequence_sum"][0])
print(f"Embedding length: {n}")

Embedding length: 512


In [33]:
embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")


Most common sequence length: 512
Count of rows with this length: 3670


In [34]:
# Function to handle the trimming or padding of the sequence
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * 512
)

print(embeded_sum["embeded_sequence_sum"].head())


0    [189.39352416992188, -147.6497344970703, -362....
1    [3.5205957889556885, 0.23739111423492432, -6.1...
2    [3.8761911392211914, 4.7096405029296875, -14.2...
3    [20.053325653076172, -9.636232376098633, -25.6...
4    [188.6451416015625, -144.83970642089844, -355....
Name: embeded_sequence_sum, dtype: object


In [35]:
embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
print(embed_lengths.value_counts())

embeded_sequence_sum
512    3694
Name: count, dtype: int64


In [36]:
# Create column names for embeddings
columns = [f'emb_{i+1}' for i in range(n)]

# Create a DataFrame for embedding values
data = pd.DataFrame(embeded_sum["embeded_sequence_sum"].to_list(), columns=columns)

# Add metadata columns
data['label'] = embeded_sum['label']
data['sample_id'] = embeded_sum['sample_id']
data['severity'] = embeded_sum['severity']

# Display the result
print(data.head())


        emb_1       emb_2       emb_3       emb_4      emb_5      emb_6  \
0  189.393524 -147.649734 -362.955963  139.653641 -85.471199 -66.070679   
1    3.520596    0.237391   -6.190857    3.774145   1.713395  -4.886091   
2    3.876191    4.709641  -14.207751    1.015683  -2.791239  -4.798626   
3   20.053326   -9.636232  -25.695652    0.598159   1.013416  -7.998230   
4  188.645142 -144.839706 -355.567810  147.655869 -95.026077 -88.978615   

        emb_7       emb_8       emb_9      emb_10  ...     emb_506    emb_507  \
0 -474.727875  120.052200  622.299561  116.579880  ... -333.419464 -25.977442   
1  -11.591867    6.478320   14.700498    2.681563  ...   -7.203462   0.416520   
2  -17.614422    5.810671   18.924332    1.923258  ...  -14.868880  -5.807928   
3  -30.077368   17.036211   28.593739   18.520885  ...  -17.841141   1.948726   
4 -476.859650  130.700760  622.824158  118.304016  ... -338.199280  -6.570933   

     emb_508     emb_509    emb_510    emb_511     emb_512  la

In [37]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codeT5_embeddings/processed_embeddings/codeT5_small_embed_line_by_line_sum.pkl'
pd.to_pickle(data, dataset_path)

In [38]:
print(embeded_avg["embeded_sequence_avg"].head())
print(embeded_avg["embeded_sequence_avg"].apply(type).value_counts())

0    0      0.203212
1     -0.158422
2     -0.38943...
1    0      0.153069
1      0.010321
2     -0.26916...
2    0      0.104762
1      0.127288
2     -0.38399...
3    0      0.294902
1     -0.141709
2     -0.37787...
4    0      0.202409
1     -0.155407
2     -0.38151...
Name: embeded_sequence_avg, dtype: object
embeded_sequence_avg
<class 'pandas.core.series.Series'>    3694
Name: count, dtype: int64


In [39]:
n = len(embeded_avg["embeded_sequence_avg"][0])
print(f"Embedding length: {n}")

Embedding length: 512


In [40]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [41]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(parse_flat_embedding)

In [42]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")

Most common sequence length: 512
Count of rows with this length: 3018


In [43]:
# Function to handle the trimming or padding of the sequence
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * 512
)

print(embeded_avg["embeded_sequence_avg"].head())

0    [0.2032119333744049, -0.15842245519161224, -0....
1    [0.1530693769454956, 0.010321352630853653, -0....
2    [0.10476192086935043, 0.1272875815629959, -0.3...
3    [0.29490184783935547, -0.14170929789543152, -0...
4    [0.20240895450115204, -0.15540741384029388, -0...
Name: embeded_sequence_avg, dtype: object


In [44]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
print(embed_lengths.value_counts())

embeded_sequence_avg
512    3694
Name: count, dtype: int64


In [45]:
columns = [f'emb_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_avg["embeded_sequence_avg"].to_list(), columns=columns)
data['label'] = embeded_avg['label']
data['sample_id'] = embeded_avg['sample_id']
data['severity'] = embeded_avg['severity']
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511,emb_512,label,sample_id,severity
0,0.203212,-0.158422,-0.389438,0.149843,-0.091707,-0.070891,-0.509365,0.128811,0.667703,0.125086,...,-0.357746,-0.027873,-0.060680,-0.374299,-0.006711,0.017262,0.171716,1,8077,critical
1,0.153069,0.010321,-0.269168,0.164093,0.074495,-0.212439,-0.503994,0.281666,0.639152,0.116590,...,0.197503,-0.313194,0.018110,-0.075032,-0.256461,-0.257803,0.129895,1,5553,critical
2,0.104762,0.127288,-0.383993,0.027451,-0.075439,-0.129693,-0.476065,0.157045,0.511468,0.051980,...,0.075900,-0.401862,-0.156971,-0.051620,-0.359596,-0.073411,0.087994,1,9341,critical
3,0.294902,-0.141709,-0.377877,0.008796,0.014903,-0.117621,-0.442314,0.250533,0.420496,0.272366,...,0.165051,-0.262370,0.028658,-0.063762,-0.251821,0.021107,0.076305,1,10419,critical
4,0.202409,-0.155407,-0.381511,0.158429,-0.101959,-0.095471,-0.511652,0.140237,0.668266,0.126936,...,-0.362875,-0.007050,-0.052576,-0.390970,-0.020881,0.020677,0.152538,1,12232,critical


In [46]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codeT5_embeddings/processed_embeddings/codeT5_small_embed_line_by_line_avg.pkl'
pd.to_pickle(data, dataset_path)